# N08. Estudio del paquete Xlwings
En este documento se muestran los ejemplos del estudio del paquete **xlwings**.

## 8.1. Uso de 'view' de xlwings



In [32]:
#*******************************************************************************
# 8.1. Carga los paquetes requeridos.
#*******************************************************************************
import numpy as np
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
import xlwings as xw
%matplotlib inline

In [2]:
#*******************************************************************************
# 8.2. Lee variables para realizar proyección de pbm.
#*******************************************************************************
aportes_m = pd.read_parquet('parquet/aportes_m.parquet.gzip', \
    engine='fastparquet')
pbhL = pd.read_parquet('parquet/pbhL.parquet.gzip', \
    engine='fastparquet')
pbm = pd.read_parquet('parquet/pbm.parquet.gzip', \
    engine='fastparquet')

In [4]:
#*******************************************************************************
# 8.3. Mira el DataFrame en Excel.
#*******************************************************************************
xw.view(pbhL) #¡Muy bien, muy bueno!

In [6]:
#*******************************************************************************
# 8.4. Mira otro DataFrame en Excel.
#*******************************************************************************
xw.view(aportes_m) # Se pueden ver varios DataFrame con el mismo objeto.

## 8.2. El modelo de objetos Excel
Los libros, las hojas y las celdas son varios de los componentes de la aplicación Excel. Estos componentes está organizados en una estructura jerárquica llamada **El modelo de objetos Excel**. Esta jerarquía está conformada por **colecciones** como 'aplicaciones', 'libros', 'hojas', 'rangos', que a su vez contienen **objetos** como 'aplicación', 'libro', 'hoja' y 'rango', entre otros.

'xlwings' usa 'app' en vez de 'application', 'book' en vez de 'workbook', 'sheet' en vez de 'worksheet'.

Una 'app' contiene la colección de 'books', un 'book' contiene la colección de 'sheets' y una 'sheet' da acceso a objetos 'range' y a otras colecciones tales como 'charts', 'names' y 'pictures'. Un 'range' contiene una o más celdas contiguas como sus elementos.

Una 'app' corresponde a una instancia o corrida del del programa Excel, es decir una aplicación que corre como un proceso separado.

Para abrir un libro se usan los siguientes comandos:

* xw.Book(). Retorna un nuevo objeto 'book'. Si no hay una instancia de excel activa, esto iniciará el programa Excel.
* xw.Book("Book1"). Retorna un objeto 'book' representando un libro de trabajo no salvado con nombre 'Book1' (el nombre del archivo no debe tener extensión).
* xw.Book("Book1.xlsx"). Retorna un objeto 'book' representando un libro de trabajo previamente salvado con el nombre 'Book1.xlsx' (el nombre del archivo debe tener extensión). Este archivo debe estar abierto o estar ubicado en el actual directorio.
* xw.Book(r"C:\path\Book1.xlsx"). Retorna un objeto 'book' representando un libro de trabajo previamente salvado con el nombre del archivo conteniendo la ruta completa. Este archivo puede estar abierto o cerrado. El prefijo  

In [2]:
#*******************************************************************************
# 8.5. Lee el libro "xlsx/Proyección demanda energía CCA 2022 - 2029.xlsx".
#*******************************************************************************
libro = xw.Book(r"xlsx/Proyección demanda energía CCA 2022 - 2029.xlsx")

In [6]:
#*******************************************************************************
# 8.6. Accesa la colección de hojas del libro.
#*******************************************************************************
libro.sheets #Solo tiene una hoja de datos: Proyeccion Consumo.

Sheets([<Sheet [Proyección demanda energía CCA 2022 - 2029.xlsx]Proyeccion Consumo>])

In [10]:
#*******************************************************************************
# 8.7. Obtiene el objeto sheet por índice o por nombre.
#*******************************************************************************
hoja = libro.sheets[0]
# hoja = libro.sheets['Proyeccion Consumo']
hoja.name

'Proyeccion Consumo'

In [16]:
#*******************************************************************************
# 8.8. Lee rango de celdas en hoja.
#*******************************************************************************
# hoja.range("A1:D156").value

## 8.3. Corre el código VBA desde Python

In [7]:
#*******************************************************************************
# 8.9. Lee el libro con código VBA.
#*******************************************************************************
libro_vba = xw.Book("xlsx/vba1.xlsm")

In [21]:
#*******************************************************************************
# 8.10. Usa una función macro VBA de Excel desde Python.
#*******************************************************************************
# Instancia el objeto macro con la función VBA.
mi_suma = libro_vba.macro("Módulo1.MySum")
# Llama una función VBA
mi_suma(5, 4)

9.0

In [12]:
#*******************************************************************************
# 8.11. Usa una subrutina macro VBA de Excel desde Python.
#*******************************************************************************
# Instancia el objeto macro con la función VBA.
# muestra_mensaje = libro_vba.macro("Módulo1.ShowMsgBox")
# Llama una función VBA
muestra_mensaje = libro_vba.macro("Módulo1.ShowMsgBox")
muestra_mensaje("Hola xlwings!")

## 8.4. Convertidores, opciones y colecciones

### 8.4.1. Trabajando con DataFrames
Escribir un DataFrame a Excel no es diferente a escribir un escalar o una lista anidada: simplemente asigne el DataFrame a la celsa superior izquierda de un rango Excel.

In [7]:
#*******************************************************************************
# 8.12. Crea libro, hoja y DataFrame.
#*******************************************************************************
libro2 = xw.Book()
libro2.sheets #Sheets([<Sheet [Libro2]Hoja1>])
hoja2 = libro2.sheets[0]
#
data = [["Mark", 55, "Italy", 4.5, "Europe"], 
        ["John", 33, "USA", 6.7, "America"]]
df = pd.DataFrame(data=data, 
                  columns=["name", "age", "country", "score", "continent"], 
                  index=[1001, 1000])
df.index.name = "user_id"
df

,name,age,country,score,continent
user_id,,,,,
1001,Mark,55,Italy,4.5,Europe
1000,John,33,USA,6.7,America


In [18]:
#*******************************************************************************
# 8.13. Escribe DataFrame en hoja Excel.
#*******************************************************************************
# hoja2["A6"].value = df # No suprime encabezado, ni índice.
# Suprime encabezado e índice.
hoja2["B10"].options(header=False, index=False).value = df

In [19]:
#*******************************************************************************
# 8.14. Lee hoja Excel y escribe en DataFrame.
#*******************************************************************************
#expand() permite leer celdas contiguas.
df2 = hoja2["A6"].expand().options(pd.DataFrame).value
df2

,name,age,country,score,continent
user_id,,,,,
1001.0,Mark,55.0,Italy,4.5,Europe
1000.0,John,33.0,USA,6.7,America


In [20]:
#*******************************************************************************
# 8.15. Convierte el índice a entero.
#*******************************************************************************
df2.index = df2.index.astype(int)
df2

,name,age,country,score,continent
user_id,,,,,
1001,Mark,55.0,Italy,4.5,Europe
1000,John,33.0,USA,6.7,America


In [21]:
#*******************************************************************************
# 8.16. Lee todas las columnas como datos, sin índices.
#*******************************************************************************
hoja2["A6"].expand().options(pd.DataFrame, index=False).value

,user_id,name,age,country,score,continent
0,1001.0,Mark,55.0,Italy,4.5,Europe
1,1000.0,John,33.0,USA,6.7,America


### 8.4.2. Convertidores y opciones
La sintasis es: myrange.options(convert=None, option1=value1, option2=value2, ...).value

* Los convertidores internos ('build-in') son:

1. dict
2. np.array
3. pd.Series
4. pd.DataFrame

* Las opciones disponibles son las siguientes:
1. empty
2. date
3. numbers
4. ndim
5. transpose
6. index
7. header

## 8.5. Gráficos (charts de Excel), pinturas (pictures de Matplotlib) y Nombres (de rangos Excel)

### 8.5.1. Gráficos de Excel

In [ ]:
#*******************************************************************************
# 8.17. Gráficos (charts) de Excel.
#*******************************************************************************


### 8.5.2. Pinturas: Gráficos (plot) de Matplotlib

In [ ]:
#*******************************************************************************
# 8.18. Gráficos (plot) de Matplotlib.
#*******************************************************************************

### 8.5.3. Nombres (de Excel)

In [15]:
#*******************************************************************************
# 8.19. Asigna nombres a varios rangos.
#*******************************************************************************
hoja2["A1:B2"].name = "matriz1"
hoja2["B10:E11"].name = "Hoja1!matriz2"
# hoja2["matriz2"]
hoja2.names
#[<Name 'Hoja1!matriz2': =Hoja1!$B$10:$E$11>]
# libro2.names 
#[<Name 'matriz1': =Hoja1!$A$1:$B$2>, <Name 'Hoja1!matriz2': =Hoja1!$B$10:$E$11>]
libro2.names["matriz1"].refers_to_range

<Range [Libro1]Hoja1!$A$1:$B$2>

In [16]:
#*******************************************************************************
# 8.20. Asigna nombres a una constante o fórmula.
#*******************************************************************************
libro2.names.add("TRM", "= 4320")

<Name 'TRM': = 4320>

## 8.6. Lee la demanda de energia mensual (demanda2)

In [24]:
#*******************************************************************************
# 8.21. Elabora df de demanda, de 2022-12-01 a 2028-12-01.
#*******************************************************************************
libro = xw.Book(r"xlsx/Proyección demanda energía CCA 2022 - 2029.xlsx")
hoja = libro.sheets[0]
demanda = hoja.range("A1:D156").options(pd.DataFrame).value
demanda = demanda.loc['2022-12-01':'2028-12-01', ['Demanda_proyectada']]
demanda.columns = ['demanda']
demanda.index.name = 'Date'
demanda = demanda/1e6

In [25]:
demanda

,demanda
Date,
2022-12-01,11.331583
2023-01-01,11.513704
2023-02-01,10.349815
2023-03-01,11.573830
2023-04-01,11.322396
...,...
2028-08-01,8.491694
2028-09-01,8.218765
2028-10-01,8.493629


In [27]:
#*******************************************************************************       
# 8.22. Salva y lee demanda del bloque.
#*******************************************************************************
# #Salva demanda
# demanda.to_parquet('parquet/demanda.parquet.gzip', \
#         compression='gzip', engine='fastparquet')
# demanda.to_excel('xlsx/demanda.xlsx', sheet_name='hoja_1')
#Lee demanda
demanda = pd.read_parquet('parquet/demanda.parquet.gzip', \
    engine='fastparquet')